YOLOV5 사용하기

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# GPU 사용 여부 확인
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# 현재 작업 디렉토리를 새로운 경로로 변경
import os

path = "/content/drive/MyDrive/Colab Notebooks/fox_star"

os.chdir(path)

HOME = os.getcwd()
print(HOME)

/content/drive/MyDrive/Colab Notebooks/fox_star


In [ ]:
# # 데이터 파일 다운 받기
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="") # API_KEY 값 제거
# project = rf.workspace("wildfire-xrkxf").project("try-tjq8g")
# version = project.version(1)
# dataset = version.download("yolov5")

In [ ]:
# YOLO 다운로드 받기
!pip install ultralytics

from IPython import display # 불필요한 정보 제거 
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 29.8/225.8 GB disk)


In [ ]:
# 불러오기
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
# 모델 내려받기
# yaml의 scale 제대로 인식 못하였지만 nano로 추측하여 scale 불러옴
# 학습된 모델이 아닌 모델 구조를 불러와서 사용

model = YOLO('/content/drive/MyDrive/Colab Notebooks/fox_star/yolov5-p6.yaml')

model.info()

WARNING ⚠️ no model scale passed. Assuming scale='n'.
YOLOv5-p6 summary: 344 layers, 4134136 parameters, 4134120 gradients, 7.3 GFLOPs


(344, 4134136, 4134120, 7.2859008)

In [ ]:
# 모델 내려받기

# model = YOLO('yolov5nu.pt')

# model.info()

YOLOv5n summary: 262 layers, 2654816 parameters, 0 gradients, 7.8 GFLOPs


(262, 2654816, 0, 7.840102399999999)

In [ ]:
# MS.COCO Dataset에 정의되어 있는 Class 개수와 종류 확인
print(type(model.names), len(model.names))

print(model.names)

<class 'dict'> 2
{0: '0', 1: '1'}


In [ ]:
# 학습데이터 선정
import yaml

data = { "train" : "/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/wild_fire/train/images/", # train_data_set
        "valid" : "/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/wild_fire/valid/images/", # val_data_set
         "test" : "/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/wild_fire/test/images/", # test_data_set
         "names" : ["fire", "smoke"], # class_name
         "nc" : 2 } # class_num

In [ ]:
# data.yaml 파일 확인
# 다른거 사용시 경로 수정해주면 됩니당
!cat '/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/wild_fire_v5/data.yaml'

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['fire', 'smoke']

roboflow:
  workspace: wildfire-xrkxf
  project: try-tjq8g
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/wildfire-xrkxf/try-tjq8g/dataset/1

In [ ]:
# local로 불러와서 UTF로 변환하기
import locale
def getpreferrdencoding(do_setlocale = True):
  return "UTF-8"
locale.getpreferredencoding = getpreferrdencoding

In [ ]:
# encoding 여부 확인
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
# 모델 학습
# Detection 할 때 UTF로 되어 있으면 py파일 자체를 못불러옴 -> 그래서 로컬로 불러와서 UTF로 변환하고 출력

model.train(data="/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/wild_fire_v5/data.yaml",
            task="detect",
            epochs=20,
            patience=30,
            imgsz=416, # stride 값 문제로 416 -> 448
            seed=42,
            project="/content/drive/MyDrive/Colab Notebooks/fox_star",
            name="star_train_yolov5n_6_Standard",
            plots=True)

Ultralytics YOLOv8.2.15 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/Colab Notebooks/fox_star/yolov5-p6.yaml, data=/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/wild_fire_v5/data.yaml, epochs=20, time=None, patience=30, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Colab Notebooks/fox_star, name=star_train_yolov5n_6_Standard, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=Fals

100%|██████████| 755k/755k [00:00<00:00, 127MB/s]


WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1      1760  ultralytics.nn.modules.conv.Conv             [3, 16, 6, 2, 2]              
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      4800  ultralytics.nn.modules.block.C3              [32, 32, 1]                   
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     29184  ultralytics.nn.modules.block.C3              [64, 64, 2]                   
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  3    156928  ultralytics.nn.modules.block.C3              [128, 128, 3]                 
  7                  -1  1    221

train: Scanning /content/drive/MyDrive/Colab Notebooks/fox_star/datasets/wild_fire_v5/train/labels.cache... 6825 images, 1 backgrounds, 0 corrupt: 100%|██████████| 6825/6825 [00:00<?, ?it/s]


WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1, len(boxes) = 19186. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all        323       1000      0.826       0.71      0.794      0.457



20 epochs completed in 5.466 hours.
Optimizer stripped from /content/drive/MyDrive/Colab Notebooks/fox_star/star_train_yolov5n_6_Standard/weights/last.pt, 8.6MB
Optimizer stripped from /content/drive/MyDrive/Colab Notebooks/fox_star/star_train_yolov5n_6_Standard/weights/best.pt, 8.6MB

Validating /content/drive/MyDrive/Colab Notebooks/fox_star/star_train_yolov5n_6_Standard/weights/best.pt...
Ultralytics YOLOv8.2.15 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv5-p6 summary (fused): 253 layers, 4126056 parameters, 0 gradients, 7.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:21<00:00,  1.94s/it]


                   all        323       1000      0.827       0.71      0.794      0.457
                  fire        323        677      0.824      0.743      0.805      0.409
                 smoke        323        323      0.829      0.678      0.784      0.504
Speed: 0.9ms preprocess, 40.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/fox_star/star_train_yolov5n_6_Standard


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d4bb1423d60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
# 검증하기
# model 학습 데이터(프로젝트)에 따라 경로 수정해줄 것

# 검증까지 마쳐야지 colab runtime error 발생 되어도 결과값 저장되어 있음

%cd {HOME}


!yolo task=detect mode=val model="/content/drive/MyDrive/Colab Notebooks/fox_star/star_train_yolov5n_6_Standard/weights/best.pt" data="/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_v5/data.yaml"

/content
Ultralytics YOLOv8.2.15 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv5-p6 summary (fused): 253 layers, 4126056 parameters, 0 gradients, 7.2 GFLOPs
Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 583, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 528, in val
    validator(model=self.model)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/validator.py", line 143, in __call__
    self.data = check_det_dataset(self.args.data)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/data/utils.py", line 269, in check_det_dataset
    file = check_

# 모델 경량화 진행하기
- Pruning

5/16 ver_1 90% del fps :
-> 우선 극단적으로 진행해서 성능 확인

In [ ]:
import torch
from torch.nn.utils import prune

# Pruning할 모델 선언
model = YOLO('/content/drive/MyDrive/Colab Notebooks/fox_star/star_train_yolov5n_6_Standard/weights/foxstar_v5n6_std.pt')

# Conv2d, Linear pruning 진행
# 가중치 값이 0에가 가까운 것들을 L2 norm 진행
# L1의 경우 미분 불가능한 점이 존재하여 Gradient-loss 발생 가능
# L2 norm의 경우제곱의 형태이기에 outlier에 더욱 민감하게 반응하여 일반화된 모델 생성이 가능할 것으로 생각

for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear): # layer의 weight에 대하여 L2 norm 기준 pruning 진행 

        if module.weight is not None and len(module.weight.shape) > 1:
            prune.ln_structured(module, name='weight', amount=0.9, n=2, dim=0) # amount는 전체 parameters 중 제거되는 parameters의 비율
        
        if module.bias is not None and len(module.bias.shape) > 1:
            prune.ln_structured(module, name='bias', amount=0.9, n=2, dim=0) # 이후 layer bias L2 norm 기준으로 pruning 진행

# 0으로 변경된 weight 값 remove 실행 
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear): # prune된 weight parameter에 remove 실행 
        
        prune.remove(module, 'weight') # prune된 bias parameter에 remove 실행

# pruned 된 모델 저장
torch.save(model.state_dict(), 'foxstar_v5n6_std_pruned_2.pt')
